In [2]:
import glob
import json
import pprint

In [3]:
irdata_list = glob.glob("./downloads/retriever/*train.json")
irdata_list

['./downloads/retriever/nq-train.json',
 './downloads/retriever/trivia-train.json',
 './downloads/retriever/nq-adv-hn-train.json',
 './downloads/retriever/curatedtrec-train.json',
 './downloads/retriever/webq-train.json',
 './downloads/retriever/squad1-train.json']

In [4]:
with open( irdata_list[0], "r",encoding='utf8') as fr:
    docs = json.load(fr)

In [5]:
print(f"{docs[0].keys()}\n{len(docs)}")

dict_keys(['dataset', 'question', 'answers', 'positive_ctxs', 'negative_ctxs', 'hard_negative_ctxs'])
58880


#### 키값 내용
  
dataset : 데이터 셋 정보  
question: 질문 문장 하나  
answers: mrc 최종 결과  
positive_ctxs: 답이 포함되어 있고, bm25 상위 랭크에 걸리는 결과들  
negative_ctxs: 답이 없지만 검색되어 나온 결과들. random인 것 같음. 검색 스코어 0  
hard_negative_ctxs: 검색 스코어가 높은데 답은 없는 검색 결과  

In [6]:
def check_doc(doc:dict):
    
    data = doc["dataset"]
    q = doc["question"]
    a = doc["answers"]
    pos = len(doc["positive_ctxs"])
    neg = len(doc["negative_ctxs"])
    h_neg = len(doc["hard_negative_ctxs"])
    
    print(f"{data} | {q} | {a}\npos:{pos} neg:{neg} hard neg:{h_neg}")

In [7]:
def get_datatext(idx, doc_key):
    pprint.pprint(docs[idx][doc_key])

title_score는 rank1 만 1 이고, 나머진 0. 뭘 기준으로 하는지 모르겠음. 
논문에서는 모든 passage에 제목을 [SEP] 토큰으로 이어붙였다고 함.

In [17]:
idx = 100

In [8]:
doc_key = "positive_ctxs"
docs[idx][doc_key][1]
#check_doc(docs[idx])
#(idx, "positive_ctxs")

{'title': 'History of roller derby',
 'text': 'three decades. Among these races was an 8.5-mile roller marathon organized in 1908 by a group of Chicago rink owners, a 24-hour endurance championship held in Milwaukee in 1913, and a 24-hour banked track race held at Madison Square Garden in 1914. "The New York Times" noted that the crowd at Madison Square Garden enjoyed the sudden sprints and spills during the event\'s preliminary races. The term "derby", meaning a race or multi-race event, appeared in the press as early as 1922, when the "Chicago Tribune" announced and reported on the results of two "roller derby" events held that year.',
 'score': 13.821991,
 'title_score': 0,
 'passage_id': '11105240'}

In [9]:
docs[idx][doc_key][0]

{'title': 'History of roller derby',
 'text': 'three decades. Among these races was an 8.5-mile roller marathon organized in 1908 by a group of Chicago rink owners, a 24-hour endurance championship held in Milwaukee in 1913, and a 24-hour banked track race held at Madison Square Garden in 1914. "The New York Times" noted that the crowd at Madison Square Garden enjoyed the sudden sprints and spills during the event\'s preliminary races. The term "derby", meaning a race or multi-race event, appeared in the press as early as 1922, when the "Chicago Tribune" announced and reported on the results of two "roller derby" events held that year.',
 'score': 1000,
 'title_score': 1,
 'passage_id': '11105240'}

In [18]:
check_doc(docs[idx])
get_datatext(idx, "positive_ctxs")

nq_train_psgs_w100 | when did zebra mussels come to north america | ['1988']
pos:6 neg:50 hard neg:95
[{'passage_id': '2709268',
  'score': 1000,
  'text': 'reporting having problems with their water treatment plants with '
          'the mussels attaching themselves to pipeworks. Anglian Water has '
          'estimated that it costs £500,000 per year to remove the mussels '
          'from their treatment plants. It has been argued that zebra mussels '
          'have also had an effect on fish populations, with dwindling fish '
          'populations in areas such as Salford Quays. They were first '
          'detected in Canada in the Great Lakes in 1988, in Lake St. Clair, '
          'located east/northeast of Detroit and Windsor. It is believed they '
          'were inadvertently introduced into the lakes in the ballast water '
          'of ocean-going ships traversing the St.',
  'title': 'Zebra mussel',
  'title_score': 1},
 {'passage_id': '15284508',
  'score': 16.869495,
 

완전 같은데 1000점 짜리랑 아닌 게 있음(idx 19, 1 등등)  
원래 답(주어진 passage)과 검색 결과인가? 아마도 그런듯

테스트

In [ ]:
python train_dense_encoder.py \
train_datasets=["nq-train.json", ] \
dev_datasets=[list of dev datasets, comma separated without spaces] \
train=biencoder_local \
output_dir={path to checkpoints dir}